In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from matplotlib.pyplot import plot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [2]:
output_size = 60
epochs = 700
features = 3

In [3]:
def get_data(y):
    train_size = int(len(y)*0.7)

    train = y[0:train_size]
    test = y[train_size:len(y)]
    print(f'Shape : {train.shape}')

    train_x = []
    train_y = []
    for i in range(0, train_size - features - output_size):
        tmp_x = y[i:(i+features)]
        tmp_y = y[(i+features):(i+features+output_size)]
        train_x.append(np.reshape(tmp_x, (1, features)))
        train_y.append(tmp_y)

    train_x = np.array(train_x)
    train_y = np.array(train_y)

    ########### TEST DATA ########################
    test_x = []
    test_y = []
    last = len(y) - output_size - features
    for i in range(train_size, last):
        tmp_x = y[i:(i+features)]
        tmp_y = y[(i + features):(i + features + output_size)]
        test_x.append(np.reshape(tmp_x, (1, features)))
        test_y.append(tmp_y)

    test_x = np.array(test_x)
    test_y = np.array(test_y)

    ######## Tahmin edilecek data #######################
    data_x = []
    tmp_x = y[-features:len(y)]
    data_x.append(np.reshape(tmp_x, (1, features)))
    data_x = np.array(data_x)

    return train_x, train_y, test_x, test_y, data_x 

In [4]:
period = '1d'
interval = '1'
df_btc = yf.download('BTC-USD', period='1d', interval= interval + 'm')
df_btc.drop(df_btc.tail(1).index,inplace=True)
df_btc.index = df_btc.index + pd.Timedelta(minutes=(180))
df_btc.tail(2)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-09-22 22:48:00+00:00,19401.714844,19401.714844,19401.714844,19401.714844,19401.714844,4325376
2022-09-22 22:49:00+00:00,19440.226562,19440.226562,19440.226562,19440.226562,19440.226562,17776640


In [5]:
df_btc.head(2)

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-09-22 03:00:00+00:00,18534.650391,18534.650391,18534.650391,18534.650391,18534.650391,0
2022-09-22 03:01:00+00:00,18497.251953,18497.251953,18497.251953,18497.251953,18497.251953,0


In [6]:
#raw_data = pd.read_csv('./datasets/ue128.csv', header=None, names=["i", "t", "y"])
t = np.array(df_btc.index.values)
y = np.array(df_btc.Close.values)

In [7]:
min = y.min()
max = y.max()

In [8]:
y = np.interp(y, (min, max), (-1, +1))

In [9]:
x_train, y_train, x_test, y_test, data_x = get_data(y)

Shape : (752,)


In [10]:
model = Sequential()
model.add(LSTM(100, input_shape=(1, features), return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(80))
model.add(Dense(output_size))

2022-09-22 22:54:45.508975: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-22 22:54:45.572035: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-22 22:54:45.572079: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-22 22:54:45.572956: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [11]:
model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=epochs, verbose=0)

2022-09-22 22:54:50.773145: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


In [12]:
score = model.evaluate(x_test, y_test)
print("%2s: %.2f%%" % (model.metrics_names[1], score[1]*100))
model.summary()

9/9 [==============================] - 1s 6ms/step - loss: 0.1101 - accuracy: 0.0231
accuracy: 2.31%
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 100)            41600     
                                                                 
 dropout (Dropout)           (None, 1, 100)            0         
                                                                 
 lstm_1 (LSTM)               (None, 80)                57920     
                                                                 
 dense (Dense)               (None, 60)                4860      
                                                                 
Total params: 104,380
Trainable params: 104,380
Non-trainable params: 0
_________________________________________________________________


In [13]:
data_y = model.predict(data_x)

1/1 [==============================] - 0s 482ms/step


In [14]:
result = np.interp(data_y, (-1, +1), (min, max))

In [17]:
df_test = pd.DataFrame()
for i in range(output_size):
    time_result = df_btc.index[-1] + pd.Timedelta(minutes=(i*int(interval)))
    print(f"Zaman {time_result}", result[0,i])

Zaman 2022-09-22 22:49:00+00:00 19287.410090283258
Zaman 2022-09-22 22:50:00+00:00 19275.840361751732
Zaman 2022-09-22 22:51:00+00:00 19247.44386374962
Zaman 2022-09-22 22:52:00+00:00 19254.669806316437
Zaman 2022-09-22 22:53:00+00:00 19259.760148525937
Zaman 2022-09-22 22:54:00+00:00 19237.44584864739
Zaman 2022-09-22 22:55:00+00:00 19243.194679848326
Zaman 2022-09-22 22:56:00+00:00 19233.698776343488
Zaman 2022-09-22 22:57:00+00:00 19228.27054781001
Zaman 2022-09-22 22:58:00+00:00 19212.678704758582
Zaman 2022-09-22 22:59:00+00:00 19256.410020128533
Zaman 2022-09-22 23:00:00+00:00 19220.341991015594
Zaman 2022-09-22 23:01:00+00:00 19249.627589624026
Zaman 2022-09-22 23:02:00+00:00 19230.059986453038
Zaman 2022-09-22 23:03:00+00:00 19256.018175816804
Zaman 2022-09-22 23:04:00+00:00 19246.109754792706
Zaman 2022-09-22 23:05:00+00:00 19243.3146272031
Zaman 2022-09-22 23:06:00+00:00 19224.670172831044
Zaman 2022-09-22 23:07:00+00:00 19225.387719405
Zaman 2022-09-22 23:08:00+00:00 19256.0